## **Szeliski Chapter 4: Model Fitting and Optimization**

In many situations, we are given *incomplete* data as input, such as depths at a sparse number of locations, or user scribbles suggesting how an image should be colorized or segmented. The problem of interpolating a complete image is called *scattered data interpolation*. Such techniques are widely used in computer vision, and also form the basis of most machine learning algorithms.



### **Scattered Data Interpolation**

The goal of *scattered data interpolation* is to produce a (usually continuous or smooth) function $f$ that passes through a set of data points $(\mathbf{x_k}, y_k)$. This is usually formulated using a penalty function like $E_D = \sum_k \lvert f(\mathbf{x}_k) - y_k)\rvert^2$. This is called a *regression* problem.


If the domain is two-dimensional, one approach is to *triangulate* the domain $\mathbf{x}$ using the input $\mathbf{x}_k$ as triangle vertices. Delaunay Triangulation can be used to obtain a triangulation using these vertices. Once a triangulation has been defined, it is straightforward to define a piecewise-linear interpolation function over each triangle. Such functions are usually continuous, but not $\mathcal{C}^1$. If a smoother surface is desired, we can replace the piecewise-linear functions using higher-order *splines*.


The *bias-variance tradeoff* refers to the tradeoff between using a large amount of regularization (which leads to lower variance, but high bias), and insufficient regularization (which leads to higher variance, but lower bias). 

##### **Radial Basis Functions**

In higher dimensions, it is common to use mesh-free approaches that define the desired interpolation as a weighted sum of basis functions. In machine learning, such approaches are called *kernel functions* or  *kernel regression*. 

More specifically, in such a formulation, $f$ is a weighted sum of basis functions centered at each data point $$f(\mathbf{x}) = \sum_k \mathbf{w}_k \Phi(\lvert \mathbf{x} - \mathbf{x}_k\rvert)$$ where the $\Phi$s are the *radial basis functions* or *kernels* and the $\mathbf{w}_k$s are the weights associated with each kernel. Some common basis functions include the Gaussian function $\phi(r) = e^{-\frac{r^2}{c^2}}$, the multi-quadratic functions $\phi(r) = \sqrt{r^2 + c^2}$ and $\phi(r) = \frac{1}{\sqrt{r^2 + c^2}}$, and the thin plate spline function $\phi(r) = r^2 \log r$.

We then minimize the $L_p$ penalty function along with an additional regularization term on the weights. 

Kernel regression has not been very widely used in computer vision, but they have been applied to some low-level image processing operations, including state-of-the-art handheld multi-frame super resolution (2019).

##### **Robust Data Fitting**

Instead of using $L_2$ and $L_1$ regularization, another solution is to use a *robust loss function $\rho$ which gives lower weights to larger data fitting errors, which are more likely to be outliers. Thus, the robust loss function has the form $$E_R = \sum_{k} \rho(\lvert f(\mathbf{x}_k - y_k\rvert)$$ In statistics, minimizing non-quadratic loss functions to deal with potential outlier measurements is known as *M-estimation*.

### **Variational Methods and Regularization**

We would like to quantify (and hence optimize) the amount of smoothness in an interpolation function, and have some degree of local control over where the function should be discontinuous. To do this, we use norms on function derivatives to formulate an energy minimization problem. Such techniques are referred to as *energy-based* or *optimization-based* approaches to computer vision. They are also called *variational* methods since we can use the calculus of variations to find the optimal solutions.


For one dimensional functions $f(x)$  we can define norms involving first partial derivatives $\Epsilon_1 = \int f_x(x)^2 dx$, as well as second partial derivatives $\Epsilon_2 = \int f_{xx}(x)^2dx$. Such energy measures are examples of functionals. 

In two dimensions, the corresponding smoothness functionals are $$\Epsilon_1 = \int\int \left(f_x^2(x,y) + f_y^2(x,y) \right) dx dy = \int\int \lvert \nabla f(x,y) \rvert^2 dx dy$$ and $$\Epsilon_2 = \int\int \left(f_{xx}^2(x,y) + 2f_{xy}^2(x,y) + f_{yy}^2(x,y) \right)dx dy$$

The first derivative norm is often called the *membrane*, since interpolating a sset of data points using this measure results in a tent-like structure. The second-order norm is called the *thin plate spline*, since it approximates the behavior of thin, flexible steel plates under small deformations.

#

### **Graphical Models in Computer Vision**

#### **Markov Random Fields**

In order to overcome ambiguities, we might want to include prior knowledge like depth maps in addition to images. We do this by formulating the problem as inference in a graphical model where each node corresponds to a pixel and models interactions between adjacent pixels. Probabilistic graphical models take a probabilistic view and model the *dependency structure* of the problem. This structured prediction problem is based on local constraints between random variables. This is useful when there is little training data, but sufficient prior knowledge (for example in medical data).

However many phenomena are hard to model with graphical models since the dependency structures may not be as simple as modelled. Also, large datasets cannot be exploited efficiently by such models.

Let $\phi$ be a non-negative function on a set of variables. We will refer to such functions $\phi$ as *potentials*. For a set of variables $\mathcal{X} = \{x_1,\ldots, x_D\}$, a **Markov Random Field** is a probability distribution defined as a product of potentials over the (maximal) cliques $\{\mathcal{X}_c\}$ of a specified undirected graph $\mathcal{G}$. That is, $$ p(x_1, \ldots, x_d) = \frac{1}{Z} \prod_{c \in C} \phi_c(\mathcal{X}_c)$$ Here $Z$ is just a normalizing constant to ensure that this function sums to 1. Recall that a clique in a graph is a fully connected subgraph, and a maximal clique is a clique that cannot be enlarged by adding any vertex in the graph.

If all cliques have size two, the markov field is referred to as a *pairwise markov random field*.

Markov Random Fields satisfy the *global markov property* which states that for every disjoint triple of sets of variables $(\mathcal{A, B, S})$ where $\mathcal{S}$ separ ates $\mathcal{A}$ from $\mathcal{B}$, we have the conditional independence relation $\mathcal{A} \coprod \mathcal{B} \mid \mathcal{S}$. This also  implies the *local markov property* that every vertex, when conditioned on its neighbors, is independent of the rest of the vertices.

##### **Factor Graphs**

Factor graphs distinguish different factorizations that lead to the same markov network by introducing an extra node for each factor. Given $\mathcal{X} = \{x_1,\ldots  x_D\}$ and $\{\mathcal{X}_k\}_{k=1}^K$ where $\mathcal{X}_k \subseteq \mathcal{X}$, and the factorization $f(\mathcal{X}) = \prod_{k=1}^{K} f_k(\mathcal{X}_k)$, the **factor graph** is a bipartite graph with a square node for each factor $f_k$ and a square node for each variable $x_i$. By normalizing $f$, we can obtain a distribution as before, $p(\mathcal{X}) = \frac{1}{Z} \prod_{k=1}^{K} f_k(\mathcal{X}_k)$.


##### **Belief Propagation**

We want to compute marginal probabilities from a factor graph, and the belief propagation algorithm allows efficient computation of such marginals. The algorithm os exact for linear graphs and trees, but also works well for general graphs.

Let us first look at inference in tree-structured factor graphs. We use the following recursive definitions $$\mu_{f \to x}(x) = \sum_{\mathcal{X}_f \setminus x} f(\mathcal{X}_f) \prod_{y \in \text{ne}(f) \setminus x} \mu_{y \to f}(y)$$ and $$\mu_{x \to f}(x) = \prod_{g \in \{\text{ne}(x) \setminus f\}} \mu_{g\to x}(x) $$ The recursive nature of these definitions lends itself to efficient computation using dynamic programming. 

We can describe this in the following manner: the message sent from node $v$ on an edge $e$ is the product of the local function at the vertex $v$ (which is just $1$ if the vertex is a variable node), with all the messages received at $v$ on edges other than $e$, summed over all variables not associated with $e$.

The belief propagation algorithm operates in three phases. 

1. First messages are sent from the leaves of the graph: messages from factor to variable $\mu_{f_A \to x}(x) = f_A(x)$, and messages from variable to factor $\mu_{x \to f_B}(x) = 1$.
2. In the middle phase, we compute variable to factor messages and factor to variable messages for all nodes where this is possible. This will be a sum of products, like $$\mu_{f_2 \to b}(b) = \sum_{c,d}f_2(b,c,d)\mu_{c\to f_2}(c)\mu_{d \to f_2}(d)$$ We repeat this step until we cover the graph.
3. We read off marginal distributions by taking the product of all the messages entering the node (we will need to normalize this vector).  

In large graph, we use log representation to help with numerical precision and since products will now become sums. In order to avoid large values, we also subtract the mean from the log representation $\lambda_{x \to f}$ after message update.

##### **Max-Product Algorithm**

Given a distribution $p(a,b,c,d)$, we would like to find the most likely state $a^*, b^*, c^*, d^* = \arg\max_{a,b,c,d} p(a,b,c,d)$. This is called the maximum-a-posteriori (MAP) solution. We again use the factorization structure to distribute the maximization to local computation. We then perform a similar computation to the one above.

##### **Loopy Belief Propagation**

In general graphs we have cycles, but messages are well defined for loopy graphs. We simply apply our belief propagation algorithm to loopy graphs. We lose exactness and guarantees of convergence, but this works surprisingly well for computer vision.

We choose either a random or fixed order message passing schedule. A popular choice is to calculate factor to variable messages first, and then variable to factor messages. This is then repeated for a desired number of iterations.




Priors based on markov random fields have the property that the log-likelihood can be described using local neighborhood interaction. Also, MRFs can be defined over discrete variables (such as image labels) for which regularization doesn't apply. 

Markov random fields are probabalistic models defined over two and three-dimensional pixel and voxel grids.